In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import dotenv
import os
import sys
import numpy as np
from tqdm import tqdm

In [2]:
# add root project directory
sys.path.append("../")
# get environment path file
dotenv_path = dotenv.find_dotenv()
# load environment variables
dotenv.load_dotenv(dotenv_path)

CALENDAR_FILE_PATH = os.environ.get("CALENDAR_FILE_PATH")
SALES_TRAIN_EVALUATION_FILE_PATH = os.environ.get("SALES_TRAIN_EVALUATION_FILE_PATH")
SALES_TRAIN_VALIDATION_FILE_PATH = os.environ.get("SALES_TRAIN_VALIDATION_FILE_PATH")
SAMPLE_SUBMISSION_FILE_PATH = os.environ.get("SAMPLE_SUBMISSION_FILE_PATH")
SELL_PRICES_FILE_PATH = os.environ.get("SELL_PRICES_FILE_PATH")

In [3]:
# load dataset
calendar = pd.read_csv(CALENDAR_FILE_PATH)
sell_price = pd.read_csv(SELL_PRICES_FILE_PATH)
sales_train_validation = pd.read_csv(SALES_TRAIN_VALIDATION_FILE_PATH)
calendar_ori = calendar.copy()
sell_price_ori = sell_price.copy()
sales_train_validation_ori = sales_train_validation.copy()

In [4]:
display(calendar.head())
display(sell_price.head())
display(sales_train_validation.head())

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


# Naive Baseline Forecast

Evaluate the model
1. Calculate the weight for the 12 level of aggregation
2. Perform naive forecast on the each of the level
3. Infer forecst, ground truth values, and weights for all the higher level series by aggregating
4. Calculate RMSSE for all the series
5. Multiple the weight to each repective RMSSE and sum it all up

In [5]:
calendar['d'] = calendar_ori['d'].apply(lambda x: x.split('_')[1]).astype(int)
sell_price['id'] = sell_price_ori['item_id'] + '_' + sell_price_ori['store_id'] + '_validation'

## 1. Calculate the weight for the 12 level of aggregation

In [6]:
for day in tqdm(range(1858, 1886)):
    wk_id = list(calendar[calendar['d']==day]['wm_yr_wk'])[0]  # select the wm_yr_wk
    wk_price_df = sell_price[sell_price['wm_yr_wk']==wk_id]  # filter based on wm_yr_wk
    sales_train_validation = sales_train_validation.merge(wk_price_df[['id', 'sell_price']], on=['id'], how='inner')  # merge it with the main transaction data
    sales_train_validation['unit_sales_' + str(day)] = sales_train_validation['sell_price'] * sales_train_validation['d_' + str(day)]  # calculate the total sell with the sell price on that specific day
    sales_train_validation.drop(columns=['sell_price'], inplace=True)  # drop the sell price

100%|██████████| 28/28 [00:26<00:00,  1.06it/s]


In [7]:
sales_train_validation['dollar_sales'] = sales_train_validation[[c for c in sales_train_validation.columns if c.find("unit_sales")==0]].sum(axis=1)

In [8]:
sales_train_validation.drop(columns=[c for c in sales_train_validation.columns if c.find("unit_sales")==0], inplace=True)

In [9]:
sales_train_validation['weight'] = sales_train_validation['dollar_sales'] / sales_train_validation['dollar_sales'].sum() 

In [10]:
sales_train_validation.drop(columns=['dollar_sales'], inplace=True)

In [11]:
sales_train_validation['weight'] /= 12

## 2. Infer forecast, ground truth values, and weights for all the higher level series by aggregating

In [12]:
agg_df = pd.DataFrame(sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_')==0]].sum()).transpose() # .transpose()
id_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
for col in id_cols:
    agg_df[col] = 'all'
agg_df['level'] = 1
agg_df['weight'] = 1/12
column_order = agg_df.columns

In [13]:
level_groupings = {
    2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
    6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 
    9: ["store_id", "dept_id"], 10: ["item_id"], 11: ["item_id", "state_id"] # , 12: ["item_id", "store_id"]
}

In [14]:
sales_train_validation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,5.258191e-06
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,8.123278e-07
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,1.012852e-06
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,5.591034e-06
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,2.029797e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,0,0,0,0,1,0,0,1,4.697087e-07
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.000000e+00
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,1,0,2,0,1,0,0,1,0,3.167010e-06
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,1,0,0,1,0,3,1,3,1.018536e-06


In [15]:
for level in tqdm(level_groupings):
    temp_df = sales_train_validation.groupby(by=level_groupings[level]).sum(numeric_only=True).reset_index()
    temp_df["level"] = level
    for c in column_order:
        if c not in temp_df.columns:
            temp_df[c] = 'all'
    agg_df = pd.concat([agg_df, temp_df[column_order]])

# del temp_df

100%|██████████| 10/10 [00:07<00:00,  1.34it/s]


In [16]:
print(sales_train_validation.shape[0], agg_df.shape[0], sales_train_validation.shape[0] + agg_df.shape[0])

30490 12350 42840


In [17]:
print(sales_train_validation['weight'].sum(), agg_df['weight'].sum(), sales_train_validation['weight'].sum() + agg_df['weight'].sum())

0.08333333333333334 0.9166666666666666 1.0


## 3. Top down forecast from different levels

In [18]:
sales_train_validation['last_28_days_mean'] = sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_')==0 and int(c.split('_')[1]) in range(1858, 1886)] + ['id']] \
    .set_index('id').transpose().mean().reset_index()[0]

In [19]:
sales_train_validation['last_28_days_mean']

0        1.000000
1        0.321429
2        0.535714
3        1.892857
4        1.107143
           ...   
30485    0.250000
30486    0.000000
30487    1.250000
30488    1.250000
30489    1.964286
Name: last_28_days_mean, Length: 30490, dtype: float64

In [25]:
for l in tqdm(range(1, 10)):
    # 1. Make forecast for the last 28 values
    this_level_df = agg_df[agg_df['level']==l].copy()
    for d in range(1, 29):
        this_level_df['f_'+str(l)+'_'+str(1885+d)] = this_level_df['d_'+str(1885+d-28)]

    # 2. Distribute the forecast by using the proportion mean
    important_column_ids = list(this_level_df[id_cols].columns[this_level_df[id_cols].nunique()!=1])
    this_level_df.reset_index(drop=True, inplace=True)
    for i, row in this_level_df.iterrows():
        if len(important_column_ids) == 0:
            level_mean_with_cond = this_level_df[[c for c in sales_train_validation.columns if c.find('d_') == 0 and\
                int(c.split('_')[1]) in range(1858, 1886)]].transpose().mean()[0]
            proportion = sales_train_validation['last_28_days_mean'] / level_mean_with_cond
            for d in range(1, 29):
                sales_train_validation['f_'+str(l)+'_'+str(1885+d)] = list(this_level_df['f_'+str(l)+'_'+str(1885+d)])[0] * proportion
        else:
            cond = True
            for col in important_column_ids:
                cond = cond & (sales_train_validation[col] == row[col])
            level_mean_with_cond = this_level_df[[c for c in sales_train_validation.columns if c.find('d_') == 0 and\
                int(c.split('_')[1]) in range(1858, 1886)]].transpose().mean()[i]
            proportion = sales_train_validation["last_28_days_mean"] / level_mean_with_cond 
            for d in range(1, 29):
                sales_train_validation.loc[cond, "f_" + str(l) + "_" + str(1885 + d)] = row["f_" + str(l) + "_" + str(1885 + d)] * proportion

# remake agg_df
new_agg_df = pd.DataFrame(sales_train_validation[[c for c in sales_train_validation.columns if c.find("d_") == 0 or c.find("f_") == 0]].sum()).transpose()
id_cols = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
for col in id_cols:
    new_agg_df[col] = 'all'
new_agg_df["level"] = 1
new_agg_df["weight"] = 1/12
column_order = new_agg_df.columns

for level in level_groupings:
    temp_df = sales_train_validation.groupby(by=level_groupings[level]).sum(numeric_only=True).reset_index()
    temp_df["level"] = level
    for c in column_order:
        if c not in temp_df.columns:
            temp_df[c] = 'all'
    new_agg_df = pd.concat([new_agg_df, temp_df[column_order]])
del temp_df

100%|██████████| 9/9 [00:03<00:00,  2.33it/s]


In [26]:
agg_df = new_agg_df

## 4. Calculate RMSSE for All The Series

In [27]:
train_series_cols = [c for c in sales_train_validation.columns if c.find("d_") == 0][:-28]
ground_truth_cols = [c for c in sales_train_validation.columns if c.find("d_") == 0][-28:]

forecast_cols_dict = {}
for i in range(1, 10):
    forecast_cols_dict[i] = [c for c in sales_train_validation.columns if c.find("f_"+str(i)+"_") == 0]

In [28]:
h = 28
n = 1885
# rmsse need 4 data - horizon, ts for training, ts for forecast, ts, for 
def rmsse(ground_truth, forecast, train_series, axis=1):
    assert axis==0 or axis==1
    assert type(ground_truth) == np.ndarray or type(forecast) == np.ndarray or type(train_series) == np.ndarray
    
    if axis  == 1:
        # if axis = 1 we have to make sure that data are in matrix format
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis==1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1])**2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1])**2).sum(axis=axis)
    
    return (1/h * numerator / denominator) ** 0.5

In [29]:
for i in range(1, 10):
    sales_train_validation["rmsse_" + str(i)] = rmsse(np.array(sales_train_validation[ground_truth_cols]), 
        np.array(sales_train_validation[forecast_cols_dict[i]]), np.array(sales_train_validation[train_series_cols]))
    agg_df["rmsse_" + str(i)] = rmsse(np.array(agg_df[ground_truth_cols]), 
        np.array(agg_df[forecast_cols_dict[i]]), np.array(agg_df[train_series_cols]))

In [30]:
for i in range(1, 10):
    sales_train_validation["wrmsse_" + str(i)] = sales_train_validation["weight"] * sales_train_validation["rmsse_" + str(i)]
    agg_df["wrmsse_" + str(i)] = agg_df["weight"] * agg_df["rmsse_" + str(i)]

In [31]:
for i in range(1, 10):
    print("Aggregation by level", str(i) + ":")
    print(sales_train_validation["wrmsse_" + str(i)].sum() + agg_df["wrmsse_" + str(i)].sum())
    print()

Aggregation by level 1:
0.7526969415454254

Aggregation by level 2:
0.7583278269835303

Aggregation by level 3:
0.7640636603286006

Aggregation by level 4:
0.7573295317439546

Aggregation by level 5:
0.7543085508691182

Aggregation by level 6:
0.7635048362696778

Aggregation by level 7:
0.7618576696686458

Aggregation by level 8:
0.7717987726658324

Aggregation by level 9:
0.7726889521635311

